Misma configuracion que fastai_efficientnetb4_v2 la version 2 pero con efficiententb5

In [3]:
if __name__ == '__main__':
    from fastai.vision.all import *
    from sklearn.metrics import mean_absolute_error, confusion_matrix, classification_report, balanced_accuracy_score, cohen_kappa_score
    import matplotlib.pyplot as plt
    from fastai.optimizer import Adam
    from functools import partial
    import torch
    import numpy as np
    import random

    # ===== ESTABLECER SEMILLA PARA REPRODUCIBILIDAD =====
    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

    torch.cuda.empty_cache()

    # ===== PARÁMETROS =====
    DATASET_PATH = Path('dataset_rangos_10_split')
    IMG_SIZE = (224, 224)
    BATCH_SIZE = 32
    EPOCHS = 80
    LR = 1e-3

    my_adamw = partial(Adam, 
                       lr=1e-3,
                       wd=1e-2,           # Weight decay
                       decouple_wd=True   # Decoupled = AdamW
                      )

    # ===== CUDA CHECK =====
    print("CUDA Available: ", torch.cuda.is_available())
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device: ", device)

    # ===== DIVISIÓN DE DATOS =====
    splitter = GrandparentSplitter(train_name='train', valid_name='valid')

    # ===== CARGAR DATOS =====
    dblock = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_items=get_image_files,
        splitter=splitter,  # División explícita en train/valid
        get_y=parent_label,
        item_tfms=Resize(IMG_SIZE),
        batch_tfms=aug_transforms(
            do_flip=True,
            max_rotate=10.0,
            max_zoom=1.2,
            max_warp=0.2,
            min_scale=0.8
        )
    )

    # ===== DATALOADERS =====
    # Antes de crear los dataloaders, fija la semilla con una función especial
    set_seed(SEED, reproducible=True)

    # Crear los dataloaders
    dls = dblock.dataloaders(DATASET_PATH, bs=BATCH_SIZE, num_workers=0)

    # Mostrar algunas imágenes
    dls.show_batch(max_n=9, figsize=(8, 8))

    # ===== TEST SET =====
    test_files = get_image_files(DATASET_PATH / 'test')
    test_dl = dls.test_dl(test_files)

    # ===== OBTENER PESOS POR CLASE =====
    class_counts = [max(len(list((DATASET_PATH / 'train' / cls).glob('*.jpg'))), 1) for cls in os.listdir(DATASET_PATH / 'train')]

    # Crear pesos inversamente proporcionales al número de imágenes en cada clase
    class_weights = torch.tensor([1.0 / c for c in class_counts], dtype=torch.float32)  # Mantén esto en CPU
    class_weights = class_weights.to(device)  # Muévelo a GPU después

    # ===== CREAR EL MODELO =====
    learn = vision_learner(
        dls,
        arch=models.efficientnet_b5,
        metrics=[accuracy],
        loss_func=CrossEntropyLossFlat(weight=class_weights.cpu()),
        opt_func=my_adamw
    ).to_fp16()

    # ===== ENTRENAMIENTO =====
    lr_recommended = learn.lr_find().valley
    print(f"Recommended Learning Rate: {lr_recommended}")

    learn.fit_one_cycle(
        EPOCHS,
        lr_recommended,
        cbs=[
            SaveModelCallback(monitor='accuracy', fname='best_model_b5_v2'),
            EarlyStoppingCallback(monitor='accuracy', patience=10),
            CSVLogger(fname='training_log_b5_v2.csv')
        ]
    )

    # Guardar el último modelo después de entrenar
    learn.save('last_model_b5_v2')

    # ===== EVALUACIÓN EN TEST =====
    learn.load('best_model_b5_v2')
    print("\n🔍 Evaluación en el conjunto de prueba (test):")
    test_loss, test_accuracy = learn.validate(dl=test_dl)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


    # Obtener predicciones en el test set
    preds, targs = learn.get_preds(dl=test_dl)
    pred_labels = preds.argmax(dim=1)

    # ===== MÉTRICAS PERSONALIZADAS =====
    if targs is not None:
        # 1. MAE (Mean Absolute Error)
        mae = mean_absolute_error(targs, pred_labels)
        print(f"Mean Absolute Error (MAE) en test: {mae:.4f}")

        # 2. Balanced Accuracy
        balanced_acc = balanced_accuracy_score(targs, pred_labels)
        print(f"Balanced Accuracy: {balanced_acc:.4f}")

        # 3. Cohen's Kappa
        kappa = cohen_kappa_score(targs, pred_labels)
        print(f"Cohen's Kappa: {kappa:.4f}")

        # 4. Matriz de Confusión
        cm = confusion_matrix(targs, pred_labels)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dls.vocab)
        disp.plot(cmap='Blues', xticks_rotation='vertical')
        plt.title('Confusion Matrix - Test')
        plt.show()

        # 5. Clasificación Detallada
        print("\nClasificación detallada (test):")
        print(classification_report(targs, pred_labels, target_names=dls.vocab))
    else:
        print("El conjunto de test no tiene etiquetas. Solo se generan predicciones.")

    # ===== GRAFICAR MÉTRICAS =====
    plt.figure(figsize=(12, 4))

    # Precisión
    plt.subplot(1, 2, 1)
    plt.plot(L(learn.recorder.values).itemgot(2), label='Train Accuracy')
    plt.plot(L(learn.recorder.values).itemgot(3), label='Validation Accuracy')
    plt.title('Accuracy per Epoch')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Pérdida
    plt.subplot(1, 2, 2)
    plt.plot(L(learn.recorder.values).itemgot(0), label='Train Loss')
    plt.plot(L(learn.recorder.values).itemgot(1), label='Validation Loss')
    plt.title('Loss per Epoch')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()


CUDA Available:  True
Device:  cuda


KeyboardInterrupt: 

In [ ]:
from fastai.vision.all import *
from sklearn.metrics import (mean_absolute_error, confusion_matrix, 
                             classification_report, balanced_accuracy_score, 
                             cohen_kappa_score, ConfusionMatrixDisplay)
import matplotlib.pyplot as plt

# ===== PARÁMETROS =====
DATASET_PATH = Path('dataset_rangos_10_split')  # Ruta de tu dataset
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# ===== CONFIGURAR DATALOADERS =====
splitter = GrandparentSplitter(train_name='train', valid_name='valid')

dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=splitter,
    get_y=parent_label,
    item_tfms=Resize(IMG_SIZE)
)

dls = dblock.dataloaders(DATASET_PATH, bs=BATCH_SIZE, num_workers=0)

# Mostrar algunas imágenes para verificar la carga
dls.show_batch(max_n=9, figsize=(8, 8))

# ===== CARGAR MODELO GUARDADO =====
learn = vision_learner(
    dls,
    arch=models.efficientnet_b5,
    metrics=[accuracy],
    loss_func=CrossEntropyLossFlat()
).to_fp16()

# Cargar el modelo guardado (ajusta el nombre si es diferente)
learn.load('best_model_b5_v2')

# ===== CREAR DATA LOADER PARA TEST =====
test_files = get_image_files(DATASET_PATH / 'test')
test_dl = dls.test_dl(test_files, with_labels=True)  # Asegúrate de incluir etiquetas

# ===== OBTENER PREDICCIONES =====
preds, targs = learn.get_preds(dl=test_dl)

if targs is not None:  # Si hay etiquetas en el conjunto de test
    pred_labels = preds.argmax(dim=1)

    # ===== MÉTRICAS =====
    # Precisión
    acc = (pred_labels == targs).float().mean()
    print(f"Test Accuracy: {acc:.4f}")

    # MAE (Mean Absolute Error)
    mae = mean_absolute_error(targs, pred_labels)
    print(f"Mean Absolute Error (MAE): {mae:.4f}")

    # Balanced Accuracy
    balanced_acc = balanced_accuracy_score(targs, pred_labels)
    print(f"Balanced Accuracy: {balanced_acc:.4f}")

    # Cohen's Kappa
    kappa = cohen_kappa_score(targs, pred_labels)
    print(f"Cohen's Kappa: {kappa:.4f}")

    # Matriz de confusión
    cm = confusion_matrix(targs, pred_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dls.vocab)
    disp.plot(cmap='Blues', xticks_rotation='vertical')
    plt.title('Confusion Matrix - Test')
    plt.show()

    # Clasificación detallada
    print("\nClasificación detallada (test):")
    print(classification_report(targs, pred_labels, target_names=dls.vocab))
else:  # Si no hay etiquetas, solo generar predicciones
    print("El conjunto de test no tiene etiquetas. Solo se generan predicciones.")
    print(preds)

# ===== EVALUACIÓN FINAL =====
print("\n🔍 Evaluación en el conjunto de prueba (test):")
test_loss, test_accuracy = learn.validate(dl=test_dl)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# ===== GRAFICAR MÉTRICAS =====
plt.figure(figsize=(12, 4))

# Precisión
plt.subplot(1, 2, 1)
plt.plot(L(learn.recorder.values).itemgot(2), label='Train Accuracy')
plt.plot(L(learn.recorder.values).itemgot(3), label='Validation Accuracy')
plt.title('Accuracy per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Pérdida
plt.subplot(1, 2, 2)
plt.plot(L(learn.recorder.values).itemgot(0), label='Train Loss')
plt.plot(L(learn.recorder.values).itemgot(1), label='Validation Loss')
plt.title('Loss per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()
